In [1]:
import numpy as np
import pandas as pd
import json
import os
from sklearn.model_selection import KFold

In [2]:
# ฟังก์ชันสำหรับอ่าน JSON หลายบรรทัด
def read_multiline_json(file_path):
    """
    อ่านไฟล์ JSON ที่มีหลายบรรทัดและแปลงเป็น list ของ dictionary
    """
    try:
        with open(file_path, 'r') as file:
            return [json.loads(line) for line in file]
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error reading JSON file at {file_path}: {e}")
        return []

# ฟังก์ชันสำหรับกรอง matrix (ลบทั้ง row และ column)
def filter_matrix_full(matrix, indices):
    """
    กรองข้อมูลใน matrix โดยลบทั้งแถว (row) และคอลัมน์ (column) ที่อยู่ใน indices ออก
    
    Parameters:
        matrix (np.ndarray): Matrix ที่ต้องการกรอง
        indices (list): Index ของแถวและคอลัมน์ที่จะลบ

    Returns:
        np.ndarray: Matrix ที่ถูกกรองแล้ว
    """
    try:
        matrix = np.delete(matrix, indices, axis=0)  # ลบแถว
        matrix = np.delete(matrix, indices, axis=1)  # ลบคอลัมน์
        return matrix
    except IndexError as e:
        print(f"Error filtering matrix: {e}")
        return matrix

# ฟังก์ชันสำหรับแบ่งข้อมูล 10-fold และกรอง matrix
def create_folds_and_filter_matrix(data, matrix, n_splits=10):
    """
    แบ่งข้อมูลเป็น 10-fold และกรอง matrix โดยลบแถวและคอลัมน์ที่เป็น validation test ออกในแต่ละ fold

    Parameters:
        data (pd.DataFrame): ข้อมูลต้นฉบับ
        matrix (np.ndarray): Matrix ที่ต้องการกรอง
        n_splits (int): จำนวน fold สำหรับ KFold

    Returns:
        list: รายการของ tuple (train_data, test_data, filtered_matrix)
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    folds = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]

        # กรอง matrix โดยลบทั้งแถวและคอลัมน์ที่เป็น validation test
        filtered_matrix = filter_matrix_full(matrix, test_index)

        # เก็บข้อมูล train, test และ filtered matrix
        folds.append((train_data, test_data, filtered_matrix))

    return folds

# ฟังก์ชันสำหรับบันทึกข้อมูลแต่ละ fold
def save_folds_as_json(folds, output_dir, prefix):
    """
    บันทึกข้อมูล train, test และ matrix ในแต่ละ fold เป็น JSON

    Parameters:
        folds (list): รายการของ fold (train_data, test_data, filtered_matrix)
        output_dir (str): โฟลเดอร์ที่ใช้บันทึกข้อมูล
        prefix (str): คำขึ้นต้นสำหรับชื่อไฟล์
    """
    os.makedirs(output_dir, exist_ok=True)

    for i, (train_data, test_data, filtered_matrix) in enumerate(folds):
        fold_dir = os.path.join(output_dir, f"fold_{i+1}")
        os.makedirs(fold_dir, exist_ok=True)

        # บันทึก train, test และ matrix เป็น JSON
        train_data.to_json(os.path.join(fold_dir, f"{prefix}_train.json"), orient="records", lines=True)
        test_data.to_json(os.path.join(fold_dir, f"{prefix}_test.json"), orient="records", lines=True)
        with open(os.path.join(fold_dir, f"{prefix}_filtered_matrix.json"), "w") as f:
            json.dump(filtered_matrix.tolist(), f)

In [3]:
# Paths ของไฟล์
malware_matrix_path = r'C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\02.EDIT_DISTANCE_MATRIX\malware_distance_matrix_1000.npy'
benign_matrix_path = r'C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\02.EDIT_DISTANCE_MATRIX\benign_distance_matrix.npy'

benign_train_path = r'C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\benign_train.json'
malware_train_path = r'C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\malware_train_1000.json'

# โหลดไฟล์ .npy
try:
    malware_matrix = np.load(malware_matrix_path)
    benign_matrix = np.load(benign_matrix_path)
except FileNotFoundError as e:
    print(f"Error loading matrix file: {e}")
    malware_matrix = benign_matrix = None

# อ่านไฟล์ JSON
benign_train_data = read_multiline_json(benign_train_path)
malware_train_data = read_multiline_json(malware_train_path)

# แปลงข้อมูล JSON เป็น DataFrame
benign_df = pd.DataFrame(benign_train_data)
malware_df = pd.DataFrame(malware_train_data)

# ตรวจสอบว่า matrix และ data ไม่เป็น None ก่อนดำเนินการ
if benign_matrix is not None and not benign_df.empty:
    benign_folds = create_folds_and_filter_matrix(benign_df, benign_matrix)
    save_folds_as_json(benign_folds, r'C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\03.CROSS_VALIDATION_10_FOLD\BENIGN_FOLDS', 'benign')
else:
    benign_folds = []
    print("Skipping benign data processing due to missing or empty data.")

if malware_matrix is not None and not malware_df.empty:
    malware_folds = create_folds_and_filter_matrix(malware_df, malware_matrix)
    save_folds_as_json(malware_folds, r'C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\03.CROSS_VALIDATION_10_FOLD\MALWARE_FOLDS', 'malware')
else:
    malware_folds = []
    print("Skipping malware data processing due to missing or empty data.")

# สรุปผลลัพธ์
print(f"Processed {len(benign_folds)} folds for benign data.")
print(f"Processed {len(malware_folds)} folds for malware data.")

Processed 10 folds for benign data.
Processed 10 folds for malware data.
